## OR

In [15]:
import pandas as pd

# Step 1: Define the file path
file_path = r"C:\Users\linke\OneDrive - AL-Hussien bin Abdullah Technical University\Desktop\Current research\OR Model\Price_veg_Full_data.csv"

# Step 2: Read the Excel file
df = pd.read_csv(file_path)

# Step 3: Convert each column to a list, then gather all lists into a single list
list_of_prices = [df[col].tolist() for col in df.columns]



In [26]:
import pulp
from tabulate import tabulate
n = 13  
T = 52
M = [8,8,12,10,10,8,6,10,8,10,8,8,8]
k = [1, 1,1,1,1,1,1,1,1,1,1,1,1,1,1]
P = [5,4,15,4.4,4,4.5,4.5,4.5,22,4,4.5,20,12]
w = [
 [1], #same number of K
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],   
 [1]
  ]

# Selling price per kg for each plant at each week
S = list_of_prices
#[
 # [9, 7, 6, 5, 4, 5, 5, 5, 6, 6, 5, 5],
  # [16, 13, 10, 12, 6, 5, 7, 8, 8, 6, 5, 7],
   #[12, 12, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12]
   #]
# Cost per unit to grow each plant at each week
C = [
    [0.9*1000]*T,
    [0.9*223]*T,
    [0.9*1700]*T,
    [0.9*240]*T,
    [0.9*240]*T,
    [0.9*240]*T,
    [0.9*240]*T,
    [0.9*520]*T,
    [0.9*1020]*T,
    [0.9*240]*T,
    [0.9*240]*T,
    [0.9*1020]*T,
    [0.9*650]*T
    
    ]
# Cost in dollars per unit to maintain and harvest plant i at week t.
Ch = [
    [0.1*1000]*T,
    [0.1*223]*T,
    [0.1*1700]*T,
    [0.1*240]*T,
    [0.1*240]*T,
    [0.1*240]*T,
    [0.1*240]*T,
    [0.1*520]*T,
    [0.1*1020]*T,
    [0.1*240]*T,
    [0.1*240]*T,
    [0.1*1020]*T,
    [0.1*650]*T
]
# Volume at week k for each plant
V = [

[0.32,  0.35,  0.40,  0.48,  0.55,  0.65,  0.75,  0.85], #Artichoke
  
  [0.41,	0.44,	0.47,	0.50,	0.52,	0.55,	0.58,	0.61],#spinach 2.13, 4.26],
  [0.335,0.40,0.50,0.625,0.80,1,1.3,1.6,1.90,2.50,2.50, 2.50],#Mulokia 0.504, 1.08,2.1],
  [0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80],#leek,
    
    [0.32,0.35,0.39,0.44,0.50,0.57,0.65,0.74,0.84,0.95], #Cabbage
    [0.315,0.34,0.375,0.42,0.47,0.525,0.575,0.625], #Parsely
    [0.325,0.36,0.41,0.465,0.51,0.555], #Arguly
    [0.35,0.40,0.45,0.50,0.60,0.65,0.70,0.75,0.80,0.85], #Lettuce 
    [0.31,  0.35,  0.4,  0.46,  0.52,  0.57,  0.63,  0.67], #celery
    [0.32,0.35,0.39,0.44,0.50,0.57,0.65,0.74,0.84,0.95],#Red Cabbage
    [0.315,0.34,0.375,0.42,0.47,0.525,0.575,0.625], #Minitg
    [0.32,  0.36,  0.41,  0.47,  0.57,  0.67,  0.77,  0.85], #Kale
    [0.32,  0.36,  0.41,  0.47,  0.57,  0.67,  0.77,  0.85] #Swiss Chard
]

# Demand for each plant at each week
D = [
    [10000000000]*T,
    [10000000000]*T,
    [10000000000]*T,
    [10000000000]*T,
    [10000000000]*T,
    [10000000000]*T,
    [10000000000]*T,
    [10000000000]*T,
    [10000000000]*T,
    [10000000000]*T,
    [10000000000]*T,
    [10000000000]*T,
    [10000000000]*T
]
L = 4000 # Total available planting volume

# Define the problem as a maximization problem
problem = pulp.LpProblem("Planting_and_Harvesting_Optimization", pulp.LpMaximize)
# Decision Variables (changed to continuous)
x = pulp.LpVariable.dicts("x", ((i, t) for i in range(n) for t in range(T)), lowBound=0, cat="Continuous")
y = pulp.LpVariable.dicts("y", ((i, t) for i in range(n) for t in range(T)), lowBound=0, cat="Continuous")

# Objective Function
problem += pulp.lpSum(
    [S[i][t] * P[i] * y[i, t] - C[i][t] * x[i, t]-Ch[i][t] * y[i, t] for i in range(n) for t in range(T)]
), "Total_Profit"

# Constraints
## Harvesting Constraint
for i in range(n):
    for t in range(T):
        if t >= M[i] :  # Condition for productive period
            problem += y[i, t] == pulp.lpSum(
                x[i, tau] * w[i][t - tau - M[i] ] 

                for tau in range(max(0, t - M[i]-k[i]+1), t - M[i]+1)
            ), f"Harvest_Timing_{i}_{t}"
        else :
            problem += y[i, t] == 0, f"Harvest_Timing_{i}_{t}"
        
##Demand constraint     
for i in range(n):
    for t in range(T):
        problem += y[i, t]*P[i] <= D[i][t], f"Y_leq_Demand_{i}_{t}"




# Volume constraint
for t in range(T):
    problem += pulp.lpSum(
        # First term: x(i,t) * V[i][1]
        x[i, t] * V[i][0] +
        
        # Second term: Summation over tau
        pulp.lpSum(
            x[i, tau] * V[i][len(V[i])-1] 
            for tau in range(max(0, t - M[i] - k[i] + 1), t - M[i] + 1)
        ) +
        
        # Third term: Summation over k
        pulp.lpSum(
            x[i, k] * V[i][t-k]
            for k in range(max(0, t - M[i] + 1), t)
        )
        for i in range(n)
    ) <= L, f"Volume_Constraint_t{t}"





# Solve the problem
#problem.solve()
status=problem.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=1260))
# Display message
if pulp.LpStatus[status] == "Optimal":
    print("✅ Optimal solution found.")
else:
    print(f"⚠️ No optimal solution. Status: {pulp.LpStatus[status]}")
table_data = []

# Add headers
headers = ["Variable"] + [f"Week {t}" for t in range(T)]

# Add rows for x(i,t)
for i in range(n):
    row_x = [f"x({i})"] + [x[i, t].varValue for t in range(T)]
    table_data.append(row_x)

# Add rows for y(i,t)
for i in range(n):
    row_y = [f"y({i})"] + [y[i, t].varValue for t in range(T)]
    table_data.append(row_y)


# Print the objective function value
print("Optimal Objective Function Value:", pulp.value(problem.objective))



# Print results in table format
print("Decision Variable Values:")
print(tabulate(table_data, headers=headers, tablefmt="grid"))

# Convert to DataFrame
df = pd.DataFrame(table_data, columns=headers)

# Save to CSV
output_path = r"C:\Users\linke\OneDrive - AL-Hussien bin Abdullah Technical University\Desktop\Current research\OR Model\Solution_trial1.csv"
df.to_csv(output_path, index=False)

print(f"\nTable saved to: {output_path}")
# Print all constraints
print("Constraints in the Model:")
for name, constraint in problem.constraints.items():
    print(f"{name}: {constraint}")
for var in problem.variables():
    print(f"{var.name} = {var.varValue}")


IndexError: list index out of range